In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
from random import choice
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler

In [2]:
album_key_dictionary = pickle.load(open('album_key_dictionary.pkl', 'rb'))
df_for_recommender = pickle.load(open('df_for_recommender.pkl', 'rb'))

In [3]:
df_for_recommender['duration_m'] = (df_for_recommender['duration_ms'] / 1000) /60

In [4]:
df_for_recommender.pop('duration_ms');
df_for_recommender

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,lyrics,duration_m
album_id,,,,,,,,,,,,,,
1awdBA8DLv6G5eBvdGxk8T,0.462455,0.873273,5.000000,-4.239455,0.818182,0.062891,0.009349,2.329909e-05,0.185600,0.559455,136.793000,3.818182,"In the back of my mind, well I fought, my Go...",3.741191
0XHpO9qTpqJJQwa2zFxAAE,0.566727,0.619909,5.454545,-10.143727,0.272727,0.042245,0.165232,2.814905e-01,0.204318,0.257136,139.569182,3.909091,Original video InstrumentalInstrumentalInstru...,4.584289
5bcobCRR5ovNoZWAp3iJgG,0.643357,0.482071,3.571429,-10.355214,0.571429,0.132664,0.659143,3.184946e-01,0.184200,0.362143,106.973857,4.000000,They said that they want me No matt...,2.956925
6Zho4ar8UMxJLMDpWcLGto,0.496250,0.183000,3.250000,-15.064500,1.000000,0.033725,0.780250,9.364000e-04,0.139750,0.141975,112.430250,4.000000,When you try your best but you don't succeed...,3.603104
07OjWJVHVzsbZ5ytXhj7ou,0.708667,0.620056,4.000000,-8.682722,0.611111,0.319444,0.031173,1.566667e-07,0.184667,0.522444,121.037000,3.777778,Lyrics for this song ha...,2.722968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2fYhqwDWXjbpjaIJPEfKFw,0.668250,0.525583,4.666667,-6.622000,0.666667,0.141642,0.298733,4.568407e-03,0.154742,0.388917,113.231167,3.833333,"Step up, the two of us, nobody knows us Get ...",3.430853
0g1F5eGVwX4Sxi1n8ojPkE,0.588000,0.664818,3.818182,-5.072455,0.909091,0.052591,0.188452,0.000000e+00,0.164909,0.518818,131.354909,4.000000,I couldn't help but notice you were sitting ...,3.215415
4gvQO5mEuhbMSrLIuwXkmz,0.783400,0.674300,5.900000,-7.241900,0.600000,0.124690,0.178350,1.067780e-03,0.158800,0.618000,117.293300,4.000000,"Yeh yeh, yeh yeh, yeh yeh La calle est ap...",3.031767


In [5]:
mapping_album_id = pd.Series(df_for_recommender.reset_index().index, index=df_for_recommender.reset_index().album_id)
mapping_album_id

album_id
1awdBA8DLv6G5eBvdGxk8T      0
0XHpO9qTpqJJQwa2zFxAAE      1
5bcobCRR5ovNoZWAp3iJgG      2
6Zho4ar8UMxJLMDpWcLGto      3
07OjWJVHVzsbZ5ytXhj7ou      4
                         ... 
2fYhqwDWXjbpjaIJPEfKFw    737
0g1F5eGVwX4Sxi1n8ojPkE    738
4gvQO5mEuhbMSrLIuwXkmz    739
6x4UQQ7TVOja89iXV047Zo    740
2xwFfH8fO1MRmWZbNKZVqt    741
Length: 742, dtype: int64

In [6]:
album_key = pd.DataFrame(album_key_dictionary.values(), index=album_key_dictionary.keys(), columns=['album_release'])
album_key

,album_release
3RoG4xBbhThunES2bojEm7,Reaching Out by Eleanor Arroway
1awdBA8DLv6G5eBvdGxk8T,Reach For The Sun by The Dangerous Summer
0XHpO9qTpqJJQwa2zFxAAE,Reach For Glory by Blackmill
5bcobCRR5ovNoZWAp3iJgG,Reaching For a Star by potsu
1coFYbyXU2dxO3lrUsbdIy,Reaching by Chama Wijnen
...,...
1pFaEu56zqpzSviJc3htZN,"thank u, next by Ariana Grande"
7AaqMMiYMvnMB3RcS8u3EY,Dan + Shay by Dan + Shay
1uyf3l2d4XYwiEqAb7t7fX,LAS QUE NO IBAN A SALIR by Bad Bunny
2lATw9ZAVp7ILQcOKPCPqp,All For Us (from the HBO Original Series Eupho...


### BASELINE MODEL
### Manually test albums
### HYPERPARAMETERS
#### components = 30, for baseline

In [7]:
def recommended_albums_topics(sample_album, X):
    similarity_matrix = cosine_similarity(X, X)
    
    album_index = mapping_album_id[sample_album]
    similarity_score = list(enumerate(similarity_matrix[album_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    # get score of 10 most similar movies. Ignore first movie.
    similarity_score = similarity_score[1:11]
    
    # return movie names using the mapping series
    
    album_indices = [i[0] for i in similarity_score]
    
    return (album_key['album_release'].iloc[album_indices])

In [8]:
lyrics = df_for_recommender.pop('lyrics');

In [9]:
# album_test = '5mG8oNkNKM2my7t9ywra96' #black sabbath - paranoid
album_test = '08eM9GRdr5BCCHNqS3Wwud' #The Miseducation of Lauryn Hill - Ms. Lauryn Hill

In [10]:
df_for_recommender.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_m
album_id,,,,,,,,,,,,,
1awdBA8DLv6G5eBvdGxk8T,0.462455,0.873273,5.000000,-4.239455,0.818182,0.062891,0.009349,2.329909e-05,0.185600,0.559455,136.793000,3.818182,3.741191
0XHpO9qTpqJJQwa2zFxAAE,0.566727,0.619909,5.454545,-10.143727,0.272727,0.042245,0.165232,2.814905e-01,0.204318,0.257136,139.569182,3.909091,4.584289
5bcobCRR5ovNoZWAp3iJgG,0.643357,0.482071,3.571429,-10.355214,0.571429,0.132664,0.659143,3.184946e-01,0.184200,0.362143,106.973857,4.000000,2.956925
6Zho4ar8UMxJLMDpWcLGto,0.496250,0.183000,3.250000,-15.064500,1.000000,0.033725,0.780250,9.364000e-04,0.139750,0.141975,112.430250,4.000000,3.603104
07OjWJVHVzsbZ5ytXhj7ou,0.708667,0.620056,4.000000,-8.682722,0.611111,0.319444,0.031173,1.566667e-07,0.184667,0.522444,121.037000,3.777778,2.722968


In [11]:
vc = CountVectorizer()
M = vc.fit_transform(lyrics)
words = vc.get_feature_names()
lda_baseline = LatentDirichletAllocation(n_components=30,
                               n_jobs=-1)
lda_baseline.fit(M)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=30, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [12]:
theta_30 = lda_baseline.transform(M)
phi = lda_baseline.components_

theta_30, phi = (np.around(x,2) for x in (theta_30, phi))
theta_30 = pd.DataFrame(theta_30, index=lyrics.index)
phi = pd.DataFrame(phi, columns=words)

print('Theta:')
display(theta_30)
print('Phi:')
display(phi)

Theta:


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
album_id,,,,,,,,,,,,,,,,,,,,,
1awdBA8DLv6G5eBvdGxk8T,0.0,0.11,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.13,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
0XHpO9qTpqJJQwa2zFxAAE,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.29,0.0,0.0,0.00
5bcobCRR5ovNoZWAp3iJgG,0.0,0.07,0.0,0.03,0.0,0.0,0.0,0.0,0.00,0.05,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.05
6Zho4ar8UMxJLMDpWcLGto,0.0,0.44,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
07OjWJVHVzsbZ5ytXhj7ou,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.01,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2fYhqwDWXjbpjaIJPEfKFw,0.0,0.04,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
0g1F5eGVwX4Sxi1n8ojPkE,0.0,0.16,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
4gvQO5mEuhbMSrLIuwXkmz,0.0,0.00,0.0,0.00,0.0,0.0,1.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00


Phi:


,00,000,000000,0001,000hours,000more,000triple,001,002,0022,...,zysk,zysku,zyskw,zytecki,zz,zza,zzette,zzz,zzzzs,zzzzzz
0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,4.41,0.03,0.03,0.03,0.03,0.03
1,3.62,0.03,0.03,0.03,1.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,1.03,0.03,0.03,0.03,0.03,1.03,0.03
2,0.03,220.58,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
3,0.03,283.83,0.03,0.03,0.03,1.03,1.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
4,56.49,325.43,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
5,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
6,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
7,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
8,0.03,13.27,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
9,0.03,47.62,0.03,0.03,0.03,0.03,0.03,14.62,6.33,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,90.03,0.03,0.03,0.03


In [14]:
final_df_30 = pd.concat([df_for_recommender, theta_30], axis=1)
final_df_30

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,20,21,22,23,24,25,26,27,28,29
album_id,,,,,,,,,,,,,,,,,,,,,
1awdBA8DLv6G5eBvdGxk8T,0.462455,0.873273,5.000000,-4.239455,0.818182,0.062891,0.009349,2.329909e-05,0.185600,0.559455,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
0XHpO9qTpqJJQwa2zFxAAE,0.566727,0.619909,5.454545,-10.143727,0.272727,0.042245,0.165232,2.814905e-01,0.204318,0.257136,...,0.0,0.0,0.0,0.0,0.00,0.0,0.29,0.0,0.0,0.00
5bcobCRR5ovNoZWAp3iJgG,0.643357,0.482071,3.571429,-10.355214,0.571429,0.132664,0.659143,3.184946e-01,0.184200,0.362143,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.05
6Zho4ar8UMxJLMDpWcLGto,0.496250,0.183000,3.250000,-15.064500,1.000000,0.033725,0.780250,9.364000e-04,0.139750,0.141975,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
07OjWJVHVzsbZ5ytXhj7ou,0.708667,0.620056,4.000000,-8.682722,0.611111,0.319444,0.031173,1.566667e-07,0.184667,0.522444,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2fYhqwDWXjbpjaIJPEfKFw,0.668250,0.525583,4.666667,-6.622000,0.666667,0.141642,0.298733,4.568407e-03,0.154742,0.388917,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
0g1F5eGVwX4Sxi1n8ojPkE,0.588000,0.664818,3.818182,-5.072455,0.909091,0.052591,0.188452,0.000000e+00,0.164909,0.518818,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00
4gvQO5mEuhbMSrLIuwXkmz,0.783400,0.674300,5.900000,-7.241900,0.600000,0.124690,0.178350,1.067780e-03,0.158800,0.618000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00


In [15]:
scaler = StandardScaler()
X = final_df_30.values
X_scaled = scaler.fit_transform(X)
recommended_albums_topics(album_test, X_scaled)

4p9dVvZDaZliSjTCbFRhJy                              High Off Life by Future
3adBogGLn3iPt5JWN5gBaw                              A Fine Mess by Interpol
3fBoL5DJZ81GttTIpPXFF8      En Vivo Con Banda y Norteño by Los Dos Carnales
4hxJ8XzBPYGHeHRFhEd42y                            Reach by Eyes Set To Kill
0fEO7g2c5onkaXsybEtuD2    Eternal Atake (Deluxe) - LUV vs. The World 2 b...
5N1L88hBfd0mg8z6KLzJLm                                 Bite Me by 413 Steph
2TFUA7Z6n24VvXdbs1hqfZ                             I'm A Fire by David Nail
2W49Vkri8OTFs5Gi7UWFUU                      Amor de red social by Micro TDH
4s5uzm05Xu09mBKv9E58x1                                   Brave by 413 Steph
549WK0ApX7KGRWEXdINVsK    Playero 41: Past Present & Future, Pt 1 by Var...
Name: album_release, dtype: object

# model by 30 topics end

### HYPERPARAMETERS
#### components = 10

In [16]:
lda_ten_comp = LatentDirichletAllocation(n_components=10,
                               n_jobs=-1)
lda_ten_comp.fit(M)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [17]:
theta_10 = lda_ten_comp.transform(M)
phi = lda_ten_comp.components_

theta_10, phi = (np.around(x,2) for x in (theta_10, phi))
theta_10 = pd.DataFrame(theta_10, index=lyrics.index)
phi = pd.DataFrame(phi, columns=words)

print('Theta:')
display(theta_10)
print('Phi:')
display(phi)

Theta:


,0,1,2,3,4,5,6,7,8,9
album_id,,,,,,,,,,
1awdBA8DLv6G5eBvdGxk8T,0.00,0.00,0.00,0.00,0.00,0.90,0.00,0.10,0.00,0.00
0XHpO9qTpqJJQwa2zFxAAE,0.01,0.01,0.01,0.41,0.01,0.01,0.48,0.01,0.01,0.01
5bcobCRR5ovNoZWAp3iJgG,0.00,0.00,0.00,0.00,0.02,0.84,0.00,0.12,0.00,0.02
6Zho4ar8UMxJLMDpWcLGto,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
07OjWJVHVzsbZ5ytXhj7ou,0.00,0.00,0.00,0.00,0.00,0.93,0.00,0.07,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
2fYhqwDWXjbpjaIJPEfKFw,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
0g1F5eGVwX4Sxi1n8ojPkE,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
4gvQO5mEuhbMSrLIuwXkmz,0.00,0.00,0.00,0.00,0.27,0.00,0.00,0.00,0.00,0.73


Phi:


,00,000,000000,0001,000hours,000more,000triple,001,002,0022,...,zysk,zysku,zyskw,zytecki,zz,zza,zzette,zzz,zzzzs,zzzzzz
0,3629.07,0.10,0.1,0.10,0.1,0.1,0.1,0.10,0.10,0.1,...,0.1,0.1,0.1,0.1,0.10,0.10,0.1,0.1,0.1,0.1
1,0.10,0.10,0.1,0.10,0.1,0.1,0.1,0.10,0.10,0.1,...,0.1,0.1,0.1,0.1,0.10,0.10,0.1,0.1,0.1,0.1
2,0.10,92.99,0.1,0.10,0.1,0.1,0.1,61.91,1.07,0.1,...,0.1,0.1,0.1,2.1,0.10,0.10,0.1,0.1,0.1,0.1
3,1183.09,595.83,0.1,0.10,0.1,0.1,0.1,35.29,18.13,0.1,...,0.1,0.1,0.1,0.1,0.10,1.12,0.1,0.1,0.1,0.1
4,8.17,40.84,0.1,0.10,0.1,0.1,0.1,0.10,9.10,81.1,...,1.1,1.1,0.1,0.1,1.25,1.08,0.1,0.1,0.1,0.1
5,0.10,88.17,2.1,0.10,0.1,0.1,0.1,0.10,0.10,0.1,...,0.1,0.1,0.1,0.1,18.77,0.10,0.1,0.1,1.1,0.1
6,119.85,214.65,0.1,1.55,0.1,0.1,0.1,0.10,0.10,0.1,...,0.1,0.1,0.1,0.1,0.10,0.10,0.1,0.1,0.1,0.1
7,4.32,511.11,0.1,0.10,0.1,1.1,1.1,0.10,0.10,0.1,...,0.1,0.1,1.1,0.1,1.70,0.10,90.1,7.1,0.1,3.1
8,0.10,3.11,0.1,2.65,1.1,0.1,0.1,0.10,0.10,0.1,...,0.1,0.1,0.1,0.1,3.68,0.10,0.1,0.1,0.1,0.1
9,0.10,0.10,0.1,0.10,0.1,0.1,0.1,0.10,0.10,0.1,...,0.1,0.1,0.1,0.1,0.10,0.10,0.1,0.1,0.1,0.1


In [18]:
final_df_10 = pd.concat([df_for_recommender, theta_10], axis=1)
final_df_10

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,0,1,2,3,4,5,6,7,8,9
album_id,,,,,,,,,,,,,,,,,,,,,
1awdBA8DLv6G5eBvdGxk8T,0.462455,0.873273,5.000000,-4.239455,0.818182,0.062891,0.009349,2.329909e-05,0.185600,0.559455,...,0.00,0.00,0.00,0.00,0.00,0.90,0.00,0.10,0.00,0.00
0XHpO9qTpqJJQwa2zFxAAE,0.566727,0.619909,5.454545,-10.143727,0.272727,0.042245,0.165232,2.814905e-01,0.204318,0.257136,...,0.01,0.01,0.01,0.41,0.01,0.01,0.48,0.01,0.01,0.01
5bcobCRR5ovNoZWAp3iJgG,0.643357,0.482071,3.571429,-10.355214,0.571429,0.132664,0.659143,3.184946e-01,0.184200,0.362143,...,0.00,0.00,0.00,0.00,0.02,0.84,0.00,0.12,0.00,0.02
6Zho4ar8UMxJLMDpWcLGto,0.496250,0.183000,3.250000,-15.064500,1.000000,0.033725,0.780250,9.364000e-04,0.139750,0.141975,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
07OjWJVHVzsbZ5ytXhj7ou,0.708667,0.620056,4.000000,-8.682722,0.611111,0.319444,0.031173,1.566667e-07,0.184667,0.522444,...,0.00,0.00,0.00,0.00,0.00,0.93,0.00,0.07,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2fYhqwDWXjbpjaIJPEfKFw,0.668250,0.525583,4.666667,-6.622000,0.666667,0.141642,0.298733,4.568407e-03,0.154742,0.388917,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
0g1F5eGVwX4Sxi1n8ojPkE,0.588000,0.664818,3.818182,-5.072455,0.909091,0.052591,0.188452,0.000000e+00,0.164909,0.518818,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
4gvQO5mEuhbMSrLIuwXkmz,0.783400,0.674300,5.900000,-7.241900,0.600000,0.124690,0.178350,1.067780e-03,0.158800,0.618000,...,0.00,0.00,0.00,0.00,0.27,0.00,0.00,0.00,0.00,0.73


In [19]:
scaler = StandardScaler()
X = final_df_10.values
X_scaled = scaler.fit_transform(X)
recommended_albums_topics(album_test, X_scaled)

5N1L88hBfd0mg8z6KLzJLm                                 Bite Me by 413 Steph
3fBoL5DJZ81GttTIpPXFF8      En Vivo Con Banda y Norteño by Los Dos Carnales
4s5uzm05Xu09mBKv9E58x1                                   Brave by 413 Steph
0fEO7g2c5onkaXsybEtuD2    Eternal Atake (Deluxe) - LUV vs. The World 2 b...
4hxJ8XzBPYGHeHRFhEd42y                            Reach by Eyes Set To Kill
2W49Vkri8OTFs5Gi7UWFUU                      Amor de red social by Micro TDH
2TFUA7Z6n24VvXdbs1hqfZ                             I'm A Fire by David Nail
3CQoFIPOp2wzvUxsU5QUjF                        Busenfreund by Infected World
56yYgfX6M5FlpETfyZSHkn                             Confident by Demi Lovato
2u0lcPtC2AoJ7M9JudSQjO                    Rainbow Connection by Lance Allen
Name: album_release, dtype: object